In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import pandas as pd

In [2]:
# input the labeled dataset
label_data0 = pd.read_csv('label_data_string_df/part-00000-9fb4a9ee-be78-4663-8f3a-ac8775296021-c000.csv', header = None)
label_data1 = pd.read_csv('label_data_string_df/part-00001-9fb4a9ee-be78-4663-8f3a-ac8775296021-c000.csv', header = None)
label_data2 = pd.read_csv('label_data_string_df/part-00002-9fb4a9ee-be78-4663-8f3a-ac8775296021-c000.csv', header = None)
label_data3 = pd.read_csv('label_data_string_df/part-00003-9fb4a9ee-be78-4663-8f3a-ac8775296021-c000.csv', header = None)
label_data = pd.concat([label_data0,label_data1,label_data2,label_data3],axis=0)
label_data.columns = ['Id', 'label', 'text']
label_data.dropna(inplace=True)
print(label_data)

          Id  label                                               text
0          0    0.0  donald,trump,sends,embarrass,new,year,eve,mess...
1          1    0.0  drunk,bragging,trump,staffer,start,russian,col...
2          2    0.0  sheriff,david,clarke,becomes,internet,joke,thr...
3          3    0.0  trump,obsess,even,obama,name,cod,website,image...
4          4    0.0  pope,francis,call,donald,trump,christmas,speec...
...      ...    ...                                                ...
21405  84893    1.0  u,official,step,rhetoric,russia,link,email,hac...
21406  84894    0.0  mcdonald,manager,hispanic,customer,spell,depor...
21407  84895    0.0  smug,liberal,law,professor,shut,tucker,carlson...
21408  84896    0.0  robert,parry,sort,russia,mess,consortium,news,...
21409  84897    1.0  israeli,missile,hit,military,post,near,damascu...

[84897 rows x 3 columns]


In [3]:
# input the unlabeled dataset
main_data0 = pd.read_csv('main_data_string_df/part-00000-db08faed-c3b7-4867-8399-c5eed41f7813-c000.csv', header = None)
main_data1 = pd.read_csv('main_data_string_df/part-00001-db08faed-c3b7-4867-8399-c5eed41f7813-c000.csv', header = None)
main_data2 = pd.read_csv('main_data_string_df/part-00002-db08faed-c3b7-4867-8399-c5eed41f7813-c000.csv', header = None)
main_data3 = pd.read_csv('main_data_string_df/part-00003-db08faed-c3b7-4867-8399-c5eed41f7813-c000.csv', header = None)
main_data = pd.concat([main_data0,main_data1,main_data2,main_data3],axis=0)
main_data.columns = ['Id', 'text']
print(main_data)

                                            Id  \
0     6a175f46bcd24d39b3e962ad0f29936721db70db   
1     2bdc29d12605ef9cf3f09f9875040a7113be5d5b   
2     c70e149fdd53de5e61c29281100b9de0ed268bc3   
3     7cf7c15731ac2a116dd7f629bd57ea468ed70284   
4     0206b54719c7e241ffe0ad4315b808290dbe6c0f   
...                                        ...   
2456  613e1a6e130b5a3f5df62c8fb0b73667742a43db   
2457  dedc36a34e5cb1062bf4627d314227f60cd9a708   
2458  cd8bb1ae426287f3a63c2979b3b5dfb0277b10e2   
2459  213eb9eeb5479ad2588b54b24acd53bc8ead8e8c   
2460  795f05dce10c27ff4e7b3f39ddf4e75075f5421c   

                                                   text  
0     muslim,bust,stole,million,gov,benefit,print,pa...  
1     attorney,general,loretta,lynch,plead,fifth,att...  
2     break,weiner,cooperate,fbi,hillary,email,inves...  
3     pin,drop,speech,father,daughter,kidnap,kill,is...  
4     fantastic,trump,point,plan,reform,healthcare,b...  
...                                                

In [4]:
# input the vocabulary
vocabulary = pd.read_csv('vocabulary.csv', na_filter = False)
vocabulary = vocabulary['vocabulary'].values.tolist()
#print(vocabulary)

In [5]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(vocabulary = vocabulary)
label_features = cv.fit_transform(label_data['text'])
main_features = cv.transform(main_data['text'])

In [6]:
label_features.shape

(84897, 100949)

In [7]:
# TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(vocabulary = vocabulary)
label_features_tfidf = tf_idf.fit_transform(label_data['text'])
main_features_tfidf = tf_idf.transform(main_data['text'])


### Prediction Models Training

In [7]:
from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier



from sklearn.metrics import accuracy_score

In [61]:
def Model(features,label):
    X = features
    y = label

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify = y,random_state= 42)
    print('Data Split Done')
    

    dtc = DecisionTreeClassifier()
    rfc = RandomForestClassifier()
    xgb=XGBClassifier(use_label_encoder=False,random_state=42,eval_metric='logloss')

    
    dtccv = GridSearchCV(dtc,param_grid={'max_depth':[10,50,100,1000,5000,10000,50000]},cv=5,n_jobs=-1,scoring = 'accuracy')

    rfccv = GridSearchCV(rfc,param_grid={'n_estimators':[200,500,1000,2000],
                                        'max_depth':[3,6,15,20],
                                        'random_state':[42]},cv=5,n_jobs=-1,scoring='accuracy')

    xgbcv=GridSearchCV(xgb,param_grid={'n_estimators':[100,200,500,1000],
                                    'max_depth' : [3,4,5,6]},
                                    cv=5,n_jobs=-1,scoring='accuracy')

    dtccv.fit(X_train,y_train)
    DTC_ACC= accuracy_score(y_test,dtccv.predict(X_test))
    print('DTC fit done')
    print(DTC_ACC)
    print(dtccv.best_estimator_)

    rfccv.fit(X_train,y_train)
    RF_ACC= accuracy_score(y_test,rfccv.predict(X_test))
    print('RF fit done')
    print(RF_ACC)
    print(rfccv.best_estimator_)

    xgbcv.fit(X_train,y_train)
    XGB_ACC = accuracy_score(y_test,xgbcv.predict(X_test))
    print('XGB fit done')
    print(XGB_ACC)
    

    model_list = [dtccv.best_estimator_,rfccv.best_estimator_,xgbcv.best_estimator_]
    acc_list = [DTC_ACC,RF_ACC,XGB_ACC]

    return model_list,acc_list


### CountVectorize Pipeline

In [62]:
Count_X = label_features
Count_y = label_data['label']

Count_model,Count_acc = Model(Count_X,Count_y)

Data Split Done
DTC fit done
0.8440518256772673
DecisionTreeClassifier(max_depth=1000)
RF fit done
0.7347074990184531
RandomForestClassifier(max_depth=15, n_estimators=2000, random_state=42)
XGB fit done
0.849195131527287


In [63]:
model_name = ['Decision Tree','Random Forest','XGBoost']

for key,value in zip(model_name,Count_acc):
    print(f'{key} Accuracy : {value}')


Decision Tree Accuracy : 0.8440518256772673
Random Forest Accuracy : 0.7347074990184531
XGBoost Accuracy : 0.849195131527287


In [64]:
CountVector_best_model = Count_model[Count_acc.index(max(Count_acc))]

In [65]:
# Save Model
from joblib import dump, load

dump(CountVector_best_model,f'{model_name[Count_acc.index(max(Count_acc))]}_CountVector.joblib') 

['XGBoost_CountVector.joblib']

In [66]:
CountVector_best_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

### TF-IDF Vectorizer Pipeline

In [67]:
TFIDF_X = label_features_tfidf
TFIDF_y = label_data['label']

TFIDF_model,TFIDF_acc = Model(TFIDF_X,TFIDF_y)

Data Split Done


In [ ]:
model_name = ['Decision Tree','Random Forest','XGBoost']

for key,value in zip(model_name,TFIDF_acc):
    print(f'{key} Accuracy : {value}')


In [ ]:
TFIDF_best_model = TFIDF_model[TFIDF_acc.index(max(TFIDF_acc))]
dump(TFIDF_best_model,f'{model_name[TFIDF_acc.index(max(TFIDF_acc))]}_TFIDF.joblib') 

### Load the Best Models

In [8]:
from joblib import dump, load

# Best CountVector Model
best = load('XGBoost_CountVector.joblib')
# Best TFIDF Model
# load('.joiblib')


In [9]:
y_new = best.predict(main_features)

In [10]:
pd.Series(y_new).value_counts()

0    8287
1    3390
dtype: int64

### Self Training

In [11]:
import numpy as np
from sklearn.semi_supervised import SelfTrainingClassifier

def self_train(X, model, threshold=0.75):
    """
    Input:
    X: numpy array or pandas dataframe
    model: sklearn model class
    Output:
    model: sklearn model class
    y_new: updated labels, numpy array
    """
    y = model.predict(X)
    prob = model.predict_proba(X)
    for i in range(len(prob)):
        if prob[i][0] < 0.9 and prob[i][1]<0.9:
            y[i] = -1
#   if len(pd.Series(y).value_counts()) != 3:
#     prob = model.predict_proba(X)
#     for i in range(len(prob)):
#       if prob[i][0]>0.99 or prob[i][1]>0.99:
#         y[i] = model.predict(X[i])
#       if len(pd.Series(y).value_counts()) == 3:
#         break
    st_model = SelfTrainingClassifier(model, threshold=threshold,
          criterion='threshold', k_best=10, max_iter=None, verbose=False)
    st_model.fit(X, y)
    y_new = st_model.transduction_
    return st_model, y_new

In [12]:
clf_new, y_new = self_train(main_features, best)

In [13]:
pd.Series(y_new).value_counts()

 0    7850
 1    3816
-1      11
dtype: int64

In [14]:
# Save Model
#dump(clf_new,'SelfTraining.joblib') 

['SelfTraining.joblib']

<25470x100949 sparse matrix of type '<class 'numpy.int64'>'
	with 3950893 stored elements in Compressed Sparse Row format>

0.7363824524206168

In [39]:
main_data['label'] = y_new

In [40]:
main_data

,Id,text,label
0,6a175f46bcd24d39b3e962ad0f29936721db70db,"muslim,bust,stole,million,gov,benefit,print,pa...",0
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,"attorney,general,loretta,lynch,plead,fifth,att...",0
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,"break,weiner,cooperate,fbi,hillary,email,inves...",1
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,"pin,drop,speech,father,daughter,kidnap,kill,is...",0
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,"fantastic,trump,point,plan,reform,healthcare,b...",0
...,...,...,...
2456,613e1a6e130b5a3f5df62c8fb0b73667742a43db,"tesla,earnings,smash,expectation,dramatic,chan...",1
2457,dedc36a34e5cb1062bf4627d314227f60cd9a708,"rule,ruler,world,really,work,follow,video,must...",0
2458,cd8bb1ae426287f3a63c2979b3b5dfb0277b10e2,"fact,check,trump,right,clinton,might,cause,ww,...",0
2459,213eb9eeb5479ad2588b54b24acd53bc8ead8e8c,"caught,tape,isi,destroys,iraqi,abrams,anti,tan...",0


In [42]:
raw = pd.read_csv('../combined/fake_news_dataset.csv')

In [47]:
raw

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12994,f1b5d0e44803f48732bde854a9fdf95837219b12,2,replaceme,2016-10-26T23:58:00.000+03:00,NaN,It DOES allow you to put a dog face on top of ...,english,2016-10-27T00:37:46.194+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs
12995,36011ceba3647e1bea78299b68b6fb705a1fc1ad,3,Freedumb,2016-10-27T00:02:00.000+03:00,NaN,Wait till you see what happens to the valuatio...,english,2016-10-27T00:37:46.220+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs
12996,6995d1aa9ac99926106489b14b5530e85358059a,4,major major maj...,2016-10-27T00:06:00.000+03:00,NaN,I'm waiting for the one that puts a pussy on m...,english,2016-10-27T00:37:46.244+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs
12997,7de8ae90eee164eb756db6c8a3772288e11d7a94,5,beemasters,2016-10-27T00:09:00.000+03:00,NaN,$4 Billion even after they are known to be kee...,english,2016-10-27T00:37:46.247+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs


In [68]:
main_data = main_data.reset_index(drop = True)
main_data

,Id,text,label
0,6a175f46bcd24d39b3e962ad0f29936721db70db,"muslim,bust,stole,million,gov,benefit,print,pa...",0
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,"attorney,general,loretta,lynch,plead,fifth,att...",0
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,"break,weiner,cooperate,fbi,hillary,email,inves...",1
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,"pin,drop,speech,father,daughter,kidnap,kill,is...",0
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,"fantastic,trump,point,plan,reform,healthcare,b...",0
...,...,...,...
11672,613e1a6e130b5a3f5df62c8fb0b73667742a43db,"tesla,earnings,smash,expectation,dramatic,chan...",1
11673,dedc36a34e5cb1062bf4627d314227f60cd9a708,"rule,ruler,world,really,work,follow,video,must...",0
11674,cd8bb1ae426287f3a63c2979b3b5dfb0277b10e2,"fact,check,trump,right,clinton,might,cause,ww,...",0
11675,213eb9eeb5479ad2588b54b24acd53bc8ead8e8c,"caught,tape,isi,destroys,iraqi,abrams,anti,tan...",0


In [78]:
label_raw = pd.Series(-1*np.ones(len(raw.uuid))).astype(int)
for i in range(len(label_raw)):
    new = main_data[main_data.Id ==raw.uuid[i]]
    if len(new) > 0:
        label_raw[i] = new.label
label_raw.value_counts()

 0    7850
 1    3816
-1    1333
dtype: int64

In [80]:
raw['label'] = label_raw

In [81]:
raw

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,...,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type,label
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,...,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,0
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,...,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,0
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,...,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,1
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,...,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias,0
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,...,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12994,f1b5d0e44803f48732bde854a9fdf95837219b12,2,replaceme,2016-10-26T23:58:00.000+03:00,NaN,It DOES allow you to put a dog face on top of ...,english,2016-10-27T00:37:46.194+03:00,zerohedge.com,US,...,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs,-1
12995,36011ceba3647e1bea78299b68b6fb705a1fc1ad,3,Freedumb,2016-10-27T00:02:00.000+03:00,NaN,Wait till you see what happens to the valuatio...,english,2016-10-27T00:37:46.220+03:00,zerohedge.com,US,...,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs,-1
12996,6995d1aa9ac99926106489b14b5530e85358059a,4,major major maj...,2016-10-27T00:06:00.000+03:00,NaN,I'm waiting for the one that puts a pussy on m...,english,2016-10-27T00:37:46.244+03:00,zerohedge.com,US,...,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs,-1
12997,7de8ae90eee164eb756db6c8a3772288e11d7a94,5,beemasters,2016-10-27T00:09:00.000+03:00,NaN,$4 Billion even after they are known to be kee...,english,2016-10-27T00:37:46.247+03:00,zerohedge.com,US,...,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs,-1


In [82]:
raw.to_csv('final.csv')

In [84]:
final = pd.read_csv('final.csv')
final.head()

,Unnamed: 0,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,...,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type,label
0,0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,...,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,0
1,1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,...,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,0
2,2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,...,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,1
3,3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,...,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias,0
4,4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,...,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias,0
